In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import csv
import pandas as pd
import numpy as np
import re

In [2]:
 nltk.download('punkt')
 nltk.download('stopwords')
 nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Birleştirilmiş veri setini yükle
df= pd.read_csv("../data/birlesik_siparisler.csv")

In [5]:
df.head()

,Items in order
0,"1 x Grilled Chicken Jamaican Tender, 1 x Grill..."
1,"1 x Peri Peri Fries, 1 x Fried Chicken Angara ..."
2,1 x Bone in Peri Peri Grilled Chicken
3,"1 x Fried Chicken Ghostbuster Tender, 1 x Anga..."
4,"1 x Peri Peri Krispers, 1 x Fried Chicken Anga..."


In [6]:
# Veri setini incele
print("Veri Seti Bilgisi:")
print(df.info())

print("\nİlk 5 Satır:")
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

print("\nEksik Veri Sayısı:")
print(df.isnull().sum())

Veri Seti Bilgisi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21321 entries, 0 to 21320
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Items in order  21321 non-null  object
dtypes: object(1)
memory usage: 166.7+ KB
None

İlk 5 Satır:
| Items in order                                                                          |
|:----------------------------------------------------------------------------------------|
| 1 x Grilled Chicken Jamaican Tender, 1 x Grilled Chicken Peri Peri Tangdi               |
| 1 x Peri Peri Fries, 1 x Fried Chicken Angara Tender, 1 x Grilled Chicken Angara Tender |
| 1 x Bone in Peri Peri Grilled Chicken                                                   |
| 1 x Fried Chicken Ghostbuster Tender, 1 x Angara Grilled Paneer (8 pcs)                 |
| 1 x Peri Peri Krispers, 1 x Fried Chicken Angara Tender                                 |

Eksik Veri Sayısı:
Items in order  

In [7]:
def stem_text(text):
  # Küçük harfe çevirme
  text = text.lower()
  # Noktalama işaretlerini kaldırma
  text = re.sub(r'[^\w\s]', '', text)

In [9]:
text = ' '.join(df['Items in order'].astype(str).tolist())  # Tüm siparisleri tek bir metin haline getir
sentences = sent_tokenize(text)  # Metni cümlelere ayır
print(sentences[:10])  # İlk 10 cümleyi yazdır

["1 x Grilled Chicken Jamaican Tender, 1 x Grilled Chicken Peri Peri Tangdi 1 x Peri Peri Fries, 1 x Fried Chicken Angara Tender, 1 x Grilled Chicken Angara Tender 1 x Bone in Peri Peri Grilled Chicken 1 x Fried Chicken Ghostbuster Tender, 1 x Angara Grilled Paneer (8 pcs) 1 x Peri Peri Krispers, 1 x Fried Chicken Angara Tender 1 x Grilled Chicken Jamaican Tangdi, 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Angara Grilled Chicken 1 x Fried Chicken Angara Tender, 1 x Angara Rice 1 x Grilled Chicken Peri Peri Tangdi 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Smoky Bbq Grilled Chicken 1 x Fried Chicken Peri Peri Tender 2 x Tipsy Tiger Ginger Ale, 1 x Bone in Jamaican Grilled Chicken, 1 x Bone in Smoky Bbq Grilled Chicken 1 x Bone in Angara Grilled Chicken 1 x Fried Chicken Peri Peri Tender 1 x Angara Aloo Tuk Tuki 1 x Grilled Chicken Peri Peri Tangdi 1 x Grilled Chicken Smoky BBQ Tender 1 x Grilled Chicke

In [12]:
# Stopwords listesini almak
stop_words = set(stopwords.words('turkish')) # Stopwords listesini turkce almak icin: turkish
stop_words_list = list(stop_words)
print(stop_words_list[:50])

['niye', 'mü', 'veya', 'da', 'biri', 'ne', 'nerede', 'birşey', 'çok', 'yani', 'hepsi', 'kim', 'siz', 've', 'için', 'sanki', 'bazı', 'hiç', 'gibi', 'az', 'bu', 'ise', 'aslında', 'niçin', 'tüm', 'ya', 'mu', 'birkaç', 'biz', 'nasıl', 'her', 'en', 'ile', 'belki', 'eğer', 'ama', 'acaba', 'hep', 'nerde', 'nereye', 'neden', 'de', 'hem', 'kez', 'şu', 'defa', 'şey', 'ki', 'o', 'diye']


In [13]:
# Lemmatizer ve Stemmer'ı başlat
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [14]:
# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
 tokens = word_tokenize(sentence) # Cümleyi kelimelere ayır
# Sadece harf olan kelimeleri al ve stopword'leri çıkar
 filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
 lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens] # Lemmatize etme
 stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens] # Stemleme
 return lemmatized_tokens, stemmed_tokens

In [15]:
# Her cümleyi tokenleştir, lemmatize et ve stemle
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

In [16]:
for sentence in sentences:
 lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
 tokenized_corpus_lemmatized.append(lemmatized_tokens)
 tokenized_corpus_stemmed.append(stemmed_tokens)

In [17]:
# lemmatize edılmıs cumlelerı bır csv dosyasına kaydedın.
with open("../data/lemmatized_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
 writer = csv.writer(file)
# Her cümleyi bir satır olarak yaz
 for tokens in tokenized_corpus_lemmatized:
  writer.writerow([' '.join(tokens)])

In [18]:
# Stem edılmıs cumlelerı bır csv dosyasına kaydedın.


with open("../data/stemmed_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Her cümleyi bir satır olarak yaz
    for tokens in tokenized_corpus_stemmed:
        writer.writerow([' '.join(tokens)])


In [19]:
 # İlk 5 cümleyi yazdıralım
for i in range(5):
    print(f"Cümle {i+1} - Base: {sentences[i]}")
    print(f"Cümle {i+1} - Lemmatized: {tokenized_corpus_lemmatized[i]}")
    print(f"Cümle {i+1} - Stemmed: {tokenized_corpus_stemmed[i]}")
    print("\n")


Cümle 1 - Base: 1 x Grilled Chicken Jamaican Tender, 1 x Grilled Chicken Peri Peri Tangdi 1 x Peri Peri Fries, 1 x Fried Chicken Angara Tender, 1 x Grilled Chicken Angara Tender 1 x Bone in Peri Peri Grilled Chicken 1 x Fried Chicken Ghostbuster Tender, 1 x Angara Grilled Paneer (8 pcs) 1 x Peri Peri Krispers, 1 x Fried Chicken Angara Tender 1 x Grilled Chicken Jamaican Tangdi, 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Angara Grilled Chicken 1 x Fried Chicken Angara Tender, 1 x Angara Rice 1 x Grilled Chicken Peri Peri Tangdi 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Jamaican Grilled Chicken 1 x Bone in Smoky Bbq Grilled Chicken 1 x Fried Chicken Peri Peri Tender 2 x Tipsy Tiger Ginger Ale, 1 x Bone in Jamaican Grilled Chicken, 1 x Bone in Smoky Bbq Grilled Chicken 1 x Bone in Angara Grilled Chicken 1 x Fried Chicken Peri Peri Tender 1 x Angara Aloo Tuk Tuki 1 x Grilled Chicken Peri Peri Tangdi 1 x Grilled Chicken Smoky BBQ Tender 1 x 